# Data Cleaning: Leads Analysis

This next one comes from: 
https://www.kaggle.com/datasets/ashydv/leads-dataset

by Ashish

## Cleaning Steps: 
1. Remove the pointless columns, as defined in the EDA
2. Fix the Lead Source Data - cases with google and the welearn title
3. Fix Specialization, How did you hear, and Lead Profile - change Select to nan
4. What is your current occupation - change to the 3 target parameters
5. Asymmetric Indexes - remove the first 3 characters from Low, Medium, High
6. Create functions to clean data 

In [15]:
import pandas as pd
import numpy as np

In [16]:
leads = pd.read_csv("Leads.csv")
leads.head()


,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,...,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,...,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


## Step 1: Remove the Pointless Columns
During the Exploratory Analysis, several columns were identified to have a single identical outcome to a yes/no question. While these data were indeed collected, they add no useful information to any potential model, and otherwise just take up space in a table that already has 37 columns. The columns to be removed are as follows:

- Magazine
- Receive more updates about our courses
- Update me on supply 
- Get updates on DM
- I agree to pay the amount through check 
- What matters to you most in choosing a course

The city column also seems to be obscure in relevance to the project, since they are very localized to a specific region in India, though the applicants are international. I am specifying this becuase it does not fit the same quality as the above. The above add no impact or relevance. The City may indeed influence any kind of model, but this is a factor that I believe should be ignored in this case, as city data was not collected consistently. 

- City


In [17]:
leads.drop(columns = ['Magazine', 
                      'Receive More Updates About Our Courses', 
                      'Update me on Supply Chain Content', 
                      'Get updates on DM Content', 
                      'I agree to pay the amount through cheque', 
                      'What matters most to you in choosing a course', 
                      'City'
                      ], inplace=True)
leads.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Through Recommendations,Tags,Lead Quality,Lead Profile,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,...,No,Interested in other courses,Low in Relevance,Select,02.Medium,02.Medium,15.0,15.0,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,...,No,Ringing,NaN,Select,02.Medium,02.Medium,15.0,15.0,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,...,No,Will revert after reading the email,Might be,Potential Lead,02.Medium,01.High,14.0,20.0,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,...,No,Ringing,Not Sure,Select,02.Medium,01.High,13.0,17.0,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,...,No,Will revert after reading the email,Might be,Select,02.Medium,01.High,15.0,18.0,No,Modified


## Step 2: Fix the Lead Source Data
These data contain 2 different 'spellings' of Google, one that is capitalized and the other is not. Additionally, there are two different entries for the welearn website, one is just the link to the homepage. 

Changes to be made: 
- Google and google = Google
- Welearnblog_home and welearn = welearn

To change these, I will use replace()
We expect to see 2 values for WeLearn and 2867+5 = 2873 for Google.

In [18]:
sources = {'google': 'Google', 
        'welearnblog_Home': 'WeLearn'}

leads['Lead Source'].replace(sources, inplace=True)
leads['Lead Source'].value_counts()


Google               2873
Direct Traffic       2543
Olark Chat           1755
Organic Search       1154
Reference             534
Welingak Website      142
Referral Sites        125
Facebook               55
bing                    6
Click2call              4
Press_Release           2
Social Media            2
WeLearn                 2
Live Chat               2
testone                 1
Pay per Click Ads       1
youtubechannel          1
blog                    1
NC_EDM                  1
Name: Lead Source, dtype: int64

## Step 3: Fix Specialization, How did you hear about X Education, Lead Profile
In each of these columns, an entry 'Select' is included; however, the Select shouldn't be included as the default option, it should be an nan. This indicates a problem with the way the survey data was recorded. Since the user did not actually select an entry, this is truly an unknown value; thus, nan is the most appropritate 'value' for these replacements. 

In [20]:
leads['Specialization'].replace({'Select':np.nan}, inplace=True)
leads['Specialization'].value_counts()

Finance Management                   976
Human Resource Management            848
Marketing Management                 838
Operations Management                503
Business Administration              403
IT Projects Management               366
Supply Chain Management              349
Banking, Investment And Insurance    338
Travel and Tourism                   203
Media and Advertising                203
International Business               178
Healthcare Management                159
Hospitality Management               114
E-COMMERCE                           112
Retail Management                    100
Rural and Agribusiness                73
E-Business                            57
Services Excellence                   40
Name: Specialization, dtype: int64

In [22]:
leads['How did you hear about X Education'].replace({'Select': np.nan}, inplace=True)
leads['How did you hear about X Education'].value_counts()

Online Search            808
Word Of Mouth            348
Student of SomeSchool    310
Other                    186
Multiple Sources         152
Advertisements            70
Social Media              67
Email                     26
SMS                       23
Name: How did you hear about X Education, dtype: int64

In [23]:
leads['Lead Profile'].replace({'Select': np.nan}, inplace=True)
leads['Lead Profile'].value_counts()

Potential Lead                 1613
Other Leads                     487
Student of SomeSchool           241
Lateral Student                  24
Dual Specialization Student      20
Name: Lead Profile, dtype: int64

## Fix the Current Occupations
The expectation based on the data definitions provided was that there would be 3 options: Employed, Unemployed, or Student
In reality, only Student is represented correctly. 

In [24]:
occupations = {
    'Working Professional':'Employed', 
    'Other':np.nan, 
    'Housewife':'Unemployed', 
    'Businessman':'Employed'
}

leads['What is your current occupation'].replace(occupations, inplace=True)
leads['What is your current occupation'].value_counts()

Unemployed    5610
Employed       714
Student        210
Name: What is your current occupation, dtype: int64

## Fix the Asymmetric Indexes
Each of these have both numbers and words in their classifications, such as 01.High... I'm just removing the numbers and leaving High.

In [26]:
levels = {
    '02.Medium':'Medium', 
    '01.High':'High', 
    '03.Low':'Low'
}

leads['Asymmetrique Activity Index'].replace(levels, inplace=True)
leads['Asymmetrique Profile Index'].replace(levels, inplace=True)

print(leads['Asymmetrique Activity Index'].value_counts(), leads['Asymmetrique Profile Index'].value_counts())


Medium    3839
High       821
Low        362
Name: Asymmetrique Activity Index, dtype: int64 Medium    2788
High      2203
Low         31
Name: Asymmetrique Profile Index, dtype: int64


# Create Functions for Data Cleaning Process

First, create a function to open the file from a user's specified path. This way all they will have to enter is the path, and they will be ultimately returned with a clean dataset. 

In [52]:
def Open_File(path):
    df = pd.read_csv(path)
    return df

In [53]:
path = 'Leads.csv'
df = Open_File(path)
df.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,...,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,...,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


Next, drop the unnecessary columns

In [54]:
def Scrub_Columns(df): 
    import pandas
    df.drop(columns=['Magazine',
                     'Receive More Updates About Our Courses',
                     'Update me on Supply Chain Content',
                     'Get updates on DM Content',
                     'I agree to pay the amount through cheque',
                     'What matters most to you in choosing a course',
                     'City'
                     ], inplace=True)
    return df

In [55]:
df = Scrub_Columns(df)
df.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Through Recommendations,Tags,Lead Quality,Lead Profile,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,...,No,Interested in other courses,Low in Relevance,Select,02.Medium,02.Medium,15.0,15.0,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,...,No,Ringing,NaN,Select,02.Medium,02.Medium,15.0,15.0,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,...,No,Will revert after reading the email,Might be,Potential Lead,02.Medium,01.High,14.0,20.0,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,...,No,Ringing,Not Sure,Select,02.Medium,01.High,13.0,17.0,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,...,No,Will revert after reading the email,Might be,Select,02.Medium,01.High,15.0,18.0,No,Modified


Now that the 7 columns have been removed, use the dictionaries to replace the erroneous or problematic data

In [56]:
def Mop_Data(df): 
    import numpy as np

    # Define the dictionaries needed for the changes
    sources = {'google': 'Google',
               'welearnblog_Home': 'WeLearn'
               }
    
    occupations = {
        'Working Professional': 'Employed',
        'Other': np.nan,
        'Housewife': 'Unemployed',
        'Businessman': 'Employed'
        }

    levels = {
        '02.Medium': 'Medium',
        '01.High': 'High',
        '03.Low': 'Low'
        }
    
    # Fix the Lead Source issues
    df['Lead Source'].replace(sources, inplace=True)
    
    # Change Select to nan in the 3 affected columns
    df['Specialization'].replace({'Select': np.nan}, inplace=True)
    df['How did you hear about X Education'].replace(
        {'Select': np.nan}, inplace=True) 
    df['Lead Profile'].replace(
        {'Select': np.nan}, inplace=True)
    
    # Fix Current Occupations
    df['What is your current occupation'].replace(occupations, inplace=True)

    # Fix Asymmetric Indexes
    df['Asymmetrique Activity Index'].replace(levels, inplace=True)
    df['Asymmetrique Profile Index'].replace(levels, inplace=True)

    return df



In [48]:
df = Mop_Data(df)

Last, create overall function to run through all subroutines taking the path as the input, and producing the dataframe as an output

In [57]:
def Wash_Data(path):
    df = Open_File(path)
    df = Scrub_Columns(df)
    df = Mop_Data(df)
    return df

Test it

In [59]:
path = 'Leads.csv'

leads = Wash_Data(path)
leads

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Through Recommendations,Tags,Lead Quality,Lead Profile,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.00,...,No,Interested in other courses,Low in Relevance,NaN,Medium,Medium,15.0,15.0,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.50,...,No,Ringing,NaN,NaN,Medium,Medium,15.0,15.0,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.00,...,No,Will revert after reading the email,Might be,Potential Lead,Medium,High,14.0,20.0,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.00,...,No,Ringing,Not Sure,NaN,Medium,High,13.0,17.0,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.00,...,No,Will revert after reading the email,Might be,NaN,Medium,High,15.0,18.0,No,Modified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,19d6451e-fcd6-407c-b83b-48e1af805ea9,579564,Landing Page Submission,Direct Traffic,Yes,No,1,8.0,1845,2.67,...,No,Will revert after reading the email,High in Relevance,Potential Lead,Medium,High,15.0,17.0,No,Email Marked Spam
9236,82a7005b-7196-4d56-95ce-a79f937a158d,579546,Landing Page Submission,Direct Traffic,No,No,0,2.0,238,2.00,...,No,wrong number given,Might be,Potential Lead,Medium,High,14.0,19.0,Yes,SMS Sent
9237,aac550fe-a586-452d-8d3c-f1b62c94e02c,579545,Landing Page Submission,Direct Traffic,Yes,No,0,2.0,199,2.00,...,No,invalid number,Not Sure,Potential Lead,Medium,High,13.0,20.0,Yes,SMS Sent
9238,5330a7d1-2f2b-4df4-85d6-64ca2f6b95b9,579538,Landing Page Submission,Google,No,No,1,3.0,499,3.00,...,No,NaN,NaN,NaN,Medium,Medium,15.0,16.0,No,SMS Sent


### Final Test 
Created a python file called Wash_Data.py

Restarting Kernel to test

In [3]:
from Wash_Data import *

In [4]:
path = 'Leads.csv'
df = Wash_Data(path)

df.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Through Recommendations,Tags,Lead Quality,Lead Profile,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,...,No,Interested in other courses,Low in Relevance,NaN,Medium,Medium,15.0,15.0,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,...,No,Ringing,NaN,NaN,Medium,Medium,15.0,15.0,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,...,No,Will revert after reading the email,Might be,Potential Lead,Medium,High,14.0,20.0,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,...,No,Ringing,Not Sure,NaN,Medium,High,13.0,17.0,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,...,No,Will revert after reading the email,Might be,NaN,Medium,High,15.0,18.0,No,Modified
